Implementación de un modelo de generacion de lenguaje utilizando el dataset propuesto de Twitter

El objetivo es simular la creación de un bot que escriba tweets de manera similar al del autor, utilizado para el entrenamiento.

# 0. Cargamos las librerías

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


# 1. Cargamos los tweets.

Para esta parte de la práctica, he utilizado los [tweets de Adam Savage](https://www.kaggle.com/speckledpingu/RawTwitterFeeds?select=AdamSavageTweets.csv)

In [3]:
df = pd.read_csv("./data/AdamSavage.csv", sep=',', decimal='.')

In [5]:
df.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,1h1 hour ago,787026890305314816,/donttrythis/status/787026890305314816,False,My #Totoro cosplay at #NYCC. Photo by @nchan.p...,AdamSavage
1,1,5h5 hours ago,786968323296595968,/donttrythis/status/786968323296595968,False,The original pic @RobtheBarbarian used to crea...,AdamSavage
2,2,5h5 hours ago,786966556785782784,/donttrythis/status/786966556785782784,False,This Sad Keanu photo made by @RobtheBarbarian ...,AdamSavage
3,3,1d1 day ago,786679762525847552,/donttrythis/status/786679762525847552,False,.@tom_sachs' shop cat sits on Totoro's leaf-ha...,AdamSavage
4,4,Oct 13,786605330851467264,/donttrythis/status/786605330851467264,False,Isn't the POV video awesome!? That's what it l...,AdamSavage


In [6]:
df.shape

(4872, 7)

In [7]:
df.dropna(subset=['text'], inplace=True)

In [8]:
df.shape

(4872, 7)

In [ ]:
Procesamos el texto

In [10]:
text = ''
for t in df['text']:
    text += ' ' + t



# 2. Mapeo de los caracteres



In [ ]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

# 3. Procesamiento 

In [ ]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)
vocab_size = len(characters)

# 4. Creamos nuestro modelo secuencial y lo entrenamos

In [12]:

model = Sequential()
model.add(LSTM(100, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
history = model.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=10, shuffle=True).history

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 100)          40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 102)               10302     
Total params: 131,502
Trainable params: 131,502
Non-trainable params: 0
_________________________________________________________________
Train on 534197 samples, validate on 28116 samples
Epoch 1/10
534197/534197 [==============================] - 877

# 5. Generación del texto

In [14]:
def generate_text(model, string_id):
    string_mapped = deepcopy(string_id)
    full_string = [n_to_char[value] for value in string_mapped]
    # Generating characters
    for i in range(400):
        x = np.reshape(string_mapped,(1,len(string_mapped), 1))
        x = x / float(len(characters))
        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = [n_to_char[value] for value in string_mapped]
        full_string.append(n_to_char[pred_index])
        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
    text = ""
    for char in full_string:
        text = text + char
    return text

# 6. Resultado 

In [29]:
model_results = generate_text(model, X[0])

In [30]:
print(model_results)

 My #Totoro cosplay at #NYCC. Photo by @nchan.pic.twitter.com/YR4oRe1zyQ The original pic @RobtheBar and the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee the seee


Como podemos observar, el resultado deja bastante que desear.
Algunas de las cosas que podríamos modificar para que nuestro modelo generase mejores 'twitts' son:
- Aumentar el número de epocas para que estubiera más tiempo entrenando, pero vigilando el overfitting
- Aumentar el número de neuronas
- Modificar el batch_size
- El porcentaje de dropout



# 7. Conclusiones

La generación de lenguaje natural, a parte de ser muy costosos los entrenamientos, en mi caso cada época a tardado unos 35 min y los resultados no han sido para nada utilizables.

Esta claro, que para poder tener un buen generado de textos, a parte del software, habría que entrenarlo por muchas más epocoas, intentando conseguir un accuracy del 96% para test, para poder tener un bueno generador de textos, ya que la idea es conseguir una imitación, no sólo del lenguaje natural, lo cual es fáscinante, sino que además, siga los patrones o estilo de escritura de una persona, que, almenos en mi caso, depende de si se comunica con familiares, amigos, compañeros de trabajo, adultos, niños, cambiará su registro, con lo que resulta muy complicado conseguir el objetivo planteado en este ejercicio

La verdad, todo sea dicho, es que me ha faltado tiempo para entrenar y optimizar los parametros, entiendo que si hubiera conseguido, aunque sólo hubiese sido un 50% de acuracy o 60% en test, hubiera alucinado con el resultado.

Aunque es el primer contacto que tengo en este campo, considero que, está muy presente en nuestro día a día,  y cada vez más.

Aunque sigo prefiriendo hablar con personas :)